In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, auc, precision_recall_curve, average_precision_score


from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('dataset_grade_2.csv').drop(columns= ['Unnamed: 0', 'SEQN'])
print(df.shape)
print(df.columns)

(560, 13)
Index(['angina', 'DPQ020', 'DPQ090', 'OHQ850', 'OHQ835', 'OHQ620', 'SMQ020',
       'WHD020', 'PAQ650', 'BPQ020', 'RIAGENDR', 'RIDAGEYR', 'DBQ700'],
      dtype='object')


## Data Preprocessing

In [3]:
# first we'll one hot encode categorical variables
dummy_depr = pd.get_dummies(df.DPQ020.astype(int), prefix='depr')
dummy_death = pd.get_dummies(df.DPQ090.astype(int), prefix = "death")
dummy_diet = pd.get_dummies(df.DBQ700, prefix = "diet")
# print(dummy_death)

# df.drop(columns = ['DPQ020', 'DPQ090', 'DBQ700'])

diet_columns = ['diet_1.0', 'diet_2.0', 'diet_3.0', 'diet_4.0', 'diet_5.0' ]
depr_columns = ['depr_0', 'depr_1', 'depr_2', 'depr_3']
death_columns = ['death_0', 'death_1', 'death_2', 'death_3']

for c in diet_columns:
    df[c] = dummy_diet[c]
for c in depr_columns:
    df[c] = dummy_depr[c]
for c in death_columns:
    df[c] = dummy_death[c]
    
print(df.shape)

(560, 26)


## Split dataset into X, y (and convert to NumPy Ndarray)

In [6]:
"""
Split dataset into X, y
Converted to NumPy Ndarray
"""
X = df.iloc[:, 1:]
# X = X.drop(columns = ['OHQ850', 'OHQ835', 'OHQ620'])
print(X.shape)
cols = X.columns
print(cols)
X = X.to_numpy()
y = df['angina'].to_numpy()
print(pd.DataFrame(y).value_counts())


(560, 25)
Index(['DPQ020', 'DPQ090', 'OHQ850', 'OHQ835', 'OHQ620', 'SMQ020', 'WHD020',
       'PAQ650', 'BPQ020', 'RIAGENDR', 'RIDAGEYR', 'DBQ700', 'diet_1.0',
       'diet_2.0', 'diet_3.0', 'diet_4.0', 'diet_5.0', 'depr_0', 'depr_1',
       'depr_2', 'depr_3', 'death_0', 'death_1', 'death_2', 'death_3'],
      dtype='object')
False    430
True     130
dtype: int64


## Split total dataset dataset into 80:20 shuffled split (train/test)

In [7]:
"""
Split total dataset into 80:20 split (train/test)
Shuffled
"""
X_train_validation, X_test, y_train_validation, y_test = train_test_split(X, y, test_size=0.20, random_state=59, shuffle=True, stratify=y)
# print(X_train)
# print(X_test)
# print(y_train)
# print(y_test)
print(X_train_validation.shape)
print(X_test.shape)
print(y_train_validation.shape)
print(y_test.shape)

(448, 25)
(112, 25)
(448,)
(112,)


## Upsample use SMOTE

In [8]:
from imblearn.over_sampling import SMOTE
# Resampling the minority class. The strategy can be changed as required.
# sm = SMOTE(sampling_strategy='minority', random_state=42)
# # Fit the model to generate the data.
# oversampled_X, oversampled_Y = sm.fit_resample(X_train_validation, y_train_validation)
# X_train_validation = oversampled_X
# y_train_validation = oversampled_Y
# print(pd.DataFrame(y_train_validation).value_counts())

## Hyperparameter Tuning (k-fold validation)

In [9]:
def hyperparam_tune(clf, alphas, testing, n_splits = 4, prnt=False, cols = cols, n = 0):
    N_MODELS = len(alphas)
    accuracy_scores = np.zeros((N_MODELS,))
    f1_scores = np.zeros((N_MODELS,))
    ROC_scores = np.zeros((N_MODELS,))
    kf = StratifiedKFold(n_splits=n_splits)
    
    for i, alpha in enumerate(alphas):
        average_accuracy = 0
        average_f1_score = 0
        average_roc_score = 0
        # run k_fold validation and sum performance metrics
        for train_index, test_index in kf.split(X_train_validation, y_train_validation):
            X_train, X_validation = X_train_validation[train_index], X_train_validation[test_index]
            y_train, y_validation = y_train_validation[train_index], y_train_validation[test_index]
            
            # Oversample using SMOTE
            sm = SMOTE(sampling_strategy='minority', random_state=42)
            oversampled_X, oversampled_Y = sm.fit_resample(X_train, y_train)
            
            X_train = pd.DataFrame(oversampled_X)
            X_train.columns = cols
            y_train = oversampled_Y
#             print(pd.DataFrame(y_train_validation).value_counts())

            #standardize the continuous columns
            age_m, age_sd = X_train['RIDAGEYR'].mean(), X_train['RIDAGEYR'].std()
            weight_m, weight_sd = X_train['WHD020'].mean(), X_train['WHD020'].std()
            mouth_m, mouth_sd = X_train['OHQ620'].mean(), X_train['OHQ620'].std()
            X_train['WHD020'] = (X_train['WHD020'] - weight_m)/weight_sd
            X_train['RIDAGEYR'] = (X_train['RIDAGEYR']-age_m)/age_sd
            X_train['OHQ620'] = (X_train['OHQ620']-mouth_m)/mouth_sd
#             print(X_train['RIDAGEYR'].mean(), X_train['RIDAGEYR'].std())
#             print(X_train['WHD020'].mean(), X_train['WHD020'].std())
#             print(X_train['OHQ620'].mean(), X_train['OHQ620'].std())
            X_train = X_train.to_numpy()
    
            
            #conditionals for different models to tune
            if testing == 'logistic':
                clf.C = alpha 
            elif testing == 'ridge':
                clf.alpha = alpha
            elif testing == "boost":
                clf.n_estimators = n
                clf.learning_rate = alpha
            elif testing == "tree":
                clf.max_depth = alpha
                clf.min_samples_split = n
                
            # fit model to data
            clf.fit(X_train, y_train)
            
            # predict labels on validation
            y_predictions = clf.predict(X_validation)
            
            #calculate averages
            average_accuracy = average_accuracy + accuracy_score(y_validation, y_predictions)
            average_f1_score = average_f1_score + f1_score(y_validation, y_predictions)
            average_roc_score = average_roc_score + roc_auc_score(y_validation, y_predictions)
          # divide performance metrics by n_splits to get averages
        accuracy_scores[i] = average_accuracy / n_splits
        f1_scores[i] = average_f1_score / n_splits
        ROC_scores[i] = average_roc_score / n_splits
        
        
    """
    Evalute best hyperparameter
    """
    alpha_with_max_accuracy = alphas[np.where(accuracy_scores == max(accuracy_scores))]
    alpha_with_max_f1_score = alphas[np.where(f1_scores == max(f1_scores))]
    alpha_with_max_ROC_score = alphas[np.where(ROC_scores == max(ROC_scores))]
    
        
    return {"max_acc": (alpha_with_max_accuracy[0], max(accuracy_scores)), 
            "max_f1": (alpha_with_max_f1_score[0], max(f1_scores)), 
            "max_roc": (alpha_with_max_ROC_score[0], max(ROC_scores))}
        

## Logistic Regression

In [10]:
N_MODELS = 100
alphas = np.logspace(0, 3, N_MODELS)
model = LogisticRegression(max_iter=1000000)

print(hyperparam_tune(model, alphas, 'logistic', n_splits = 4, prnt=False))

{'max_acc': (1.0, 0.6339285714285714), 'max_f1': (1.0, 0.09420289855072464), 'max_roc': (1.0, 0.5)}


## Ridge Classifier

In [16]:
from sklearn.linear_model import RidgeClassifier
alphas = np.logspace(2, 3, N_MODELS)
model = RidgeClassifier()

print(hyperparam_tune(model, alphas, 'ridge', n_splits = 4, prnt=False))

{'max_acc': (100.0, 0.6339285714285714), 'max_f1': (215.44346900318845, 0.1933764522697329), 'max_roc': (215.44346900318845, 0.5121869409660107)}


## Support Vector Classification

In [17]:
from sklearn.svm import SVC

alphas = np.logspace(0, 2, N_MODELS)
model = SVC(kernel =  'sigmoid')

print(hyperparam_tune(model, alphas, 'logistic', n_splits = 4, prnt=False))

{'max_acc': (95.45484566618342, 0.5267857142857143), 'max_f1': (3.6783797718286344, 0.3802410797378124), 'max_roc': (57.223676593502205, 0.5190071556350626)}


In [19]:
from sklearn.tree import DecisionTreeClassifier

# alphas = np.logspace(0, 0, 1)
# model = DecisionTreeClassifier()

# print(hyperparam_tune(model, alphas, 'tree', n_splits = 4, prnt=False))

## Adaboost

In [20]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

alphas = np.logspace(-6, 0, N_MODELS)
model = AdaBoostClassifier()

print(hyperparam_tune(model, alphas, 'boost', n_splits = 4, prnt=False, n = 75))

{'max_acc': (0.7564633275546291, 0.7589285714285715), 'max_f1': (1e-06, 0.31758389833548567), 'max_roc': (0.7564633275546291, 0.5277280858676208)}


## Random Forest

In [21]:
alphas = np.logspace(0, 2, 10)
model = RandomForestClassifier(max_features='log2')

print(hyperparam_tune(model, alphas, 'tree', n = 2, prnt=False))

{'max_acc': (12.91549665014884, 0.7522321428571428), 'max_f1': (1.6681005372000588, 0.23428501835516818), 'max_roc': (12.91549665014884, 0.5300760286225402)}


# Test Set Scores

## Test Random Forest

In [35]:
# X_train_validation, X_test, y_train_validation, y_test = train_test_split(X, y, test_size=0.2, random_state=59, shuffle=True, stratify=y)
# Resampling the minority class. The strategy can be changed as required.
sm = SMOTE(sampling_strategy='minority', random_state=42)
# Fit the model to generate the data.
oversampled_X, oversampled_Y = sm.fit_resample(X_train_validation, y_train_validation)
X_train_validation = oversampled_X
X_train_validation = pd.DataFrame(oversampled_X)
X_train_validation.columns = cols
y_train_validation = oversampled_Y
print(pd.DataFrame(y_train_validation).value_counts())
print(X_train_validation.shape)

#standardize the continuous columns
age_m, age_sd = X_train_validation['RIDAGEYR'].mean(), X_train_validation['RIDAGEYR'].std()
weight_m, weight_sd = X_train_validation['WHD020'].mean(), X_train_validation['WHD020'].std()
mouth_m, mouth_sd = X_train_validation['OHQ620'].mean(), X_train_validation['OHQ620'].std()
X_train_validation['WHD020'] = (X_train_validation['WHD020'] - weight_m)/weight_sd
X_train_validation['RIDAGEYR'] = (X_train_validation['RIDAGEYR']-age_m)/age_sd
X_train_validation['OHQ620'] = (X_train_validation['OHQ620']-mouth_m)/mouth_sd
#             print(X_train['RIDAGEYR'].mean(), X_train['RIDAGEYR'].std())
#             print(X_train['WHD020'].mean(), X_train['WHD020'].std())
#             print(X_train['OHQ620'].mean(), X_train['OHQ620'].std())
X_train_validation = X_train_validation.to_numpy()

model = RandomForestClassifier(max_features='log2', max_depth = 1, min_samples_split = 2)
model.fit(X_train_validation, y_train_validation)
rf_predictions = model.predict(X_test)
rf_predictions2 = model.predict(X_train_validation)
print("\nTESTING RESULTS")
print(f"Accuracy: {accuracy_score(y_test, rf_predictions)}")
print(f"F1: {f1_score(y_test, rf_predictions)}")
print(f"AUC-ROC: {roc_auc_score(y_test, rf_predictions)}")

print("\nTRAINING RESULTS")
print(f"Accuracy: {accuracy_score(y_train_validation, rf_predictions2)}")
print(f"F1: {f1_score(y_train_validation, rf_predictions2)}")
print(f"AUC-ROC: {roc_auc_score(y_train_validation, rf_predictions2)}")

print('')
print("PR-AUC:")
print("TRAINING:")
lr_probs2 = model.predict_proba(X_train_validation)[:,1]
lr_probs = model.predict_proba(X_test)[:,1]
precision, recall, thresh = precision_recall_curve(y_train_validation, lr_probs2)
# print(average_precision_score(y_train_validation, lr_probs2))
print(auc(recall, precision))
print("TESTING:")
precision, recall, thresh = precision_recall_curve(y_test, lr_probs)
print(auc(recall, precision))
# print(average_precision_score(y_test, lr_probs))


False    344
True     344
dtype: int64
(688, 25)

TESTING RESULTS
Accuracy: 0.5714285714285714
F1: 0.22580645161290322
AUC-ROC: 0.4660107334525939

TRAINING RESULTS
Accuracy: 0.747093023255814
F1: 0.7371601208459214
AUC-ROC: 0.747093023255814

PR-AUC:
TRAINING:
0.8232056211846152
TESTING:
0.2406687972392009


## Test Logistic Regression

In [36]:
model = LogisticRegression(max_iter=1000000, C=1.5199110829529336)
model.fit(X_train_validation, y_train_validation)
lr_predictions = model.predict(X_test)
lr_predictions2 = model.predict(X_train_validation)
print("\nTESTING RESULTS")
print(f"Accuracy {accuracy_score(y_test, lr_predictions)}")
print(f"F1 :{f1_score(y_test, lr_predictions)}")
print(f"AUC-ROC: {roc_auc_score(y_test, lr_predictions)}")

print("\nTRAINING RESULTS")
print(f"Accuracy {accuracy_score(y_train_validation, lr_predictions2)}")
print(f"F1 :{f1_score(y_train_validation, lr_predictions2)}")
print(f"AUC-ROC: {roc_auc_score(y_train_validation, lr_predictions2)}")

print('')
print("PR ROC AUC:")
print("TRAINING:")
lr_probs2 = model.predict_proba(X_train_validation)[:,1]
lr_probs = model.predict_proba(X_test)[:,1]
precision, recall, thresh = precision_recall_curve(y_train_validation, lr_probs2)
# print(average_precision_score(y_train_validation, lr_probs2))
print(auc(recall, precision))
print("TESTING:")
precision, recall, thresh = precision_recall_curve(y_test, lr_probs)
print(auc(recall, precision))
# print(average_precision_score(y_test, lr_predictions))


TESTING RESULTS
Accuracy 0.5446428571428571
F1 :0.2816901408450704
AUC-ROC: 0.488819320214669

TRAINING RESULTS
Accuracy 0.6366279069767442
F1 :0.6468926553672316
AUC-ROC: 0.6366279069767442

PR ROC AUC:
TRAINING:
0.6658478599226556
TESTING:
0.21576857219366682


In [37]:
model = AdaBoostClassifier(n_estimators = 75, learning_rate = 1e-06)
model.fit(X_train_validation, y_train_validation)
ab_predictions = model.predict(X_test)
ab_predictions2 = model.predict(X_train_validation)
print("\nTESTING RESULTS")
print(f"Accuracy: {accuracy_score(y_test, ab_predictions)}")
print(f"F1: {f1_score(y_test, ab_predictions)}")
print(F"AUC-ROC: {roc_auc_score(y_test, ab_predictions)}")

print("\nTRAINING RESULTS")
print(f"Accuracy: {accuracy_score(y_train_validation, ab_predictions2)}")
print(f"F1: {f1_score(y_train_validation, ab_predictions2)}")
print(F"AUC-ROC: {roc_auc_score(y_train_validation, ab_predictions2)}")
print('')
print("PR ROC AUC:")
print("TRAINING:")
lr_probs2 = model.predict_proba(X_train_validation)[:,1]
lr_probs = model.predict_proba(X_test)[:,1]
precision, recall, thresh = precision_recall_curve(y_train_validation, lr_probs2)
print(auc(recall, precision))
# print(average_precision_score(y_train_validation, lr_probs2))
print("TESTING:")
precision, recall, thresh = precision_recall_curve(y_test, lr_probs)
print(auc(recall, precision))
# print(average_precision_score(y_test, lr_probs))


TESTING RESULTS
Accuracy: 0.44642857142857145
F1: 0.35416666666666663
AUC-ROC: 0.518783542039356

TRAINING RESULTS
Accuracy: 0.6366279069767442
F1: 0.7037914691943128
AUC-ROC: 0.6366279069767442

PR ROC AUC:
TRAINING:
0.7628430232558139
TESTING:
0.48853021978021977


## Calculate Precision + Recall

In [40]:
def create_truth(row):
    if row['truth'] == 1 and row['pred'] == 1:
        return "TP"
    if row['truth'] == 1 and row['pred'] == 0:
        return "FN"
    if row['truth'] ==0  and row['pred'] == 1:
        return "FP"
    if row['truth'] == 0 and row['pred'] == 0:
        return "TN"
    

def precision_recall(dct):
    if 'FN' not in dct.keys():
        recall = dct['TP'] / (dct['TP']+0)
    else:
        recall = dct['TP'] / (dct['TP'] + dct['FN'])
        
    precision = dct['TP'] / (dct['TP'] + dct['FP'])
    return precision, recall

def create_dict(pred, truth, verbose= False):
    df = pd.DataFrame()
    df['truth'] = truth
    df['pred'] = pred
    df['result'] = df.apply(lambda row: create_truth(row), axis = 1)
    if verbose == True:
        print(df['result'].value_counts())
    return dict(df['result'].value_counts())

print("Test results:")

rf_dct = create_dict(rf_predictions, y_test)
rf_prec, rf_rec = precision_recall(rf_dct)
print(f"RandomForest precision: {rf_prec}, recall: {rf_rec}")
# print((2*(rf_prec * rf_rec))/(rf_prec + rf_rec))

svm_dct = create_dict(lr_predictions, y_test)
svm_prec, svm_rec = precision_recall(svm_dct)
print(f"LR precision: {svm_prec}, recall: {svm_rec}")

ab_dct = create_dict(ab_predictions, y_test)
ab_prec, ab_rec = precision_recall(ab_dct)
print(f"AdaBoost precision: {ab_prec}, recall: {ab_rec}")


print("\nTraining results:")
rf_dct = create_dict(rf_predictions2, y_train_validation)
rf_prec, rf_rec = precision_recall(rf_dct)
print(f"RandomForest precision: {rf_prec}, recall: {rf_rec}")
# print((2*(rf_prec * rf_rec))/(rf_prec + rf_rec))

svm_dct = create_dict(lr_predictions2,  y_train_validation)
svm_prec, svm_rec = precision_recall(svm_dct)
print(f"LR precision: {svm_prec}, recall: {svm_rec}")

ab_dct = create_dict(ab_predictions2,  y_train_validation)
ab_prec, ab_rec = precision_recall(ab_dct)
print(f"AdaBoost precision: {ab_prec}, recall: {ab_rec}")

Test results:
RandomForest precision: 0.19444444444444445, recall: 0.2692307692307692
LR precision: 0.2222222222222222, recall: 0.38461538461538464
AdaBoost precision: 0.24285714285714285, recall: 0.6538461538461539

Training results:
RandomForest precision: 0.7672955974842768, recall: 0.7093023255813954
LR precision: 0.6291208791208791, recall: 0.6656976744186046
AdaBoost precision: 0.594, recall: 0.8633720930232558
